# **Importamos librerias**


In [ ]:
! pip install optuna

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 390.6/390.6 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 19.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 4.7 MB/s eta 0:00:00


In [ ]:
import pandas as pd

import numpy as np

import optuna

import joblib

import xgboost as xgb

from sklearn.model_selection import train_test_split

from sklearn.metrics import roc_auc_score

from sklearn.metrics import accuracy_score

from sklearn.metrics import recall_score


# conectamos al drive

from google.colab import drive

# **Funciones de trabajo**

In [ ]:
def objective(trial):

    # definimos los parametros

    params_opt={

        'use_rmm':True,

        'tree_method':'gpu_hist' , # aqui activas la gpu

        'objective':'binary:logistic',

        'grow_policy':'lossguide',

        'eta':trial.suggest_float('eta',0.001,0.1,log=True) ,

        'colsample_bytree' : trial.suggest_float('colsample_bytree',0.1,1) ,

        'subsample': trial.suggest_float('subsample',0.1,1) ,

        'max_depth':trial.suggest_int('max_depth',3,100),

        #'alpha':trial.suggest_float('alpha',0.0001, 100,log=True),

        #'lambda':trial.suggest_float('lambda',0.0001, 100, log=True),

        'gamma':trial.suggest_float('gamma',0.00001,10,log=True),

        'min_child_weight':trial.suggest_float('min_child_weight',1,1000,log=True),

        'scale_pos_weight':spw

        }

    # preparamos la data

    dtrain=xgb.DMatrix(data=X_train , label=y_train)
    dtest=xgb.DMatrix(data=X_cross, label=y_cross)

    xgb_clf = xgb.train(
        params=params_opt,
        dtrain=dtrain,
        num_boost_round=1000
                        )

    y_pred=xgb_clf.predict(dtest)

    metric=roc_auc_score(y_cross,y_pred)

    return metric

# **Conectamos al Drive**

In [ ]:

# conectamos al drive

drive.mount('/content/drive')

Mounted at /content/drive


# **Importamos la data**

In [ ]:
# importamos primero el compresor y el normalizador

# para normalizar

dir_data_kaggle='./drive/MyDrive/springleaf/data'


X_cross, X_test, y_cross,y_test= train_test_split(
                                              np.load(dir_data_kaggle+'/df_X_test.npy'),
                                              np.load(dir_data_kaggle+'/df_y_test.npy'),
                                              test_size=0.5 )



# descargamos la data en el formato requerido


X_train=np.load(dir_data_kaggle+'/df_X_train.npy')

y_train=np.load(dir_data_kaggle+'/df_y_train.npy')




In [ ]:
# ajustamos el parametro scale_pos_weight

spw=len(y_train[y_train==0])/len(y_train[y_train==1])

# **Costruimos el modelo y optimizamos**

In [ ]:
#Hacemos un estudio

study_xgb = optuna.create_study(

    study_name='opt_xgb',
    direction='maximize',
    sampler=optuna.samplers.CmaEsSampler(
        x0=joblib.load(dir_data_kaggle+'/xgb_params.sav') # en la primera iteracion se definio este diccionario
    )
                        )

study_xgb.optimize(func=objective,
                   n_trials=10,
                   gc_after_trial=True,
                   timeout=7200,
                   show_progress_bar=True)

[I 2023-07-21 18:14:44,842] A new study created in memory with name: opt_xgb


  0%|          | 0/10 [00:00<?, ?it/s]

[I 2023-07-21 18:18:46,283] Trial 0 finished with value: 0.779567881323124 and parameters: {'eta': 0.0034297832489172775, 'colsample_bytree': 0.9752243801334584, 'subsample': 0.5255429643619748, 'max_depth': 96, 'gamma': 3.903293445023343e-05, 'min_child_weight': 36.94890550964283}. Best is trial 0 with value: 0.779567881323124.
[I 2023-07-21 18:27:53,750] Trial 1 finished with value: 0.7868494485443417 and parameters: {'eta': 0.01041044800422565, 'colsample_bytree': 0.6475447934574325, 'subsample': 0.8454611459757295, 'max_depth': 68, 'gamma': 0.0006593085174923201, 'min_child_weight': 8.751608909845336}. Best is trial 1 with value: 0.7868494485443417.
[I 2023-07-21 18:32:18,257] Trial 2 finished with value: 0.7874194041203779 and parameters: {'eta': 0.011666675646515341, 'colsample_bytree': 0.5914507205237207, 'subsample': 0.6992754776909421, 'max_depth': 72, 'gamma': 5.338196620322527e-05, 'min_child_weight': 24.49057862328063}. Best is trial 2 with value: 0.7874194041203779.
[I 202

In [ ]:
print(f'\nBest Params : {study_xgb.best_params}')
print(f'\nROC_AUC Best params : {study_xgb.best_value}\n')


Best Params : {'eta': 0.011666675646515341, 'colsample_bytree': 0.5914507205237207, 'subsample': 0.6992754776909421, 'max_depth': 72, 'gamma': 5.338196620322527e-05, 'min_child_weight': 24.49057862328063}

ROC_AUC Best params : 0.7874194041203779



In [ ]:
# verificamos el modelo


 # definimos los parametros optimizados

params_opt=study_xgb.best_params


params_stat={

        'use_rmm':True,

        'tree_method':'gpu_hist' , # aqui activas la gpu para el entrenamiento

        'objective':'binary:logistic',

        'grow_policy':'lossguide',

        'scale_pos_weight':spw
                                }


# juntamos ambas


params={**params_opt,**params_stat}


    # preparamos la data

dtrain=xgb.DMatrix(

    data=X_train ,

    label=y_train

                                )

dtest=xgb.DMatrix(

    data=X_test,

    label=y_test
                                )

xgb_clf = xgb.train(
        params=params,
        dtrain=dtrain,
        num_boost_round=5000
                        )





In [ ]:

y_pred=xgb_clf.predict(dtest)


metric_auc=roc_auc_score(y_test,y_pred)

metric_acc=accuracy_score(y_test, (y_pred> 0.5).astype(np.float32) )

metric_rec=recall_score(y_test, (y_pred> 0.5).astype(np.float32) )

print(f'\n\nauc={np.round(metric_auc,4)}\n\naccuracy={np.round(metric_acc,4)}\n\nrecall={np.round(metric_rec,4)}')

In [ ]:
# exportamos los parametros


joblib.dump(params,dir_data_kaggle+'/xgb_params.sav')


['./drive/MyDrive/springleaf/data/xgb_params.sav']

['./drive/MyDrive/springleaf/data/xgb_params.sav']